<a href="https://colab.research.google.com/github/utkarsh0702/Tensorflow/blob/master/tensorflow6_Horse_%26_Human_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import os
from tensorflow import keras
import zipfile

In [0]:
local_zip= '/content/horse-or-human.zip'
zip_ref= zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/horse-or-human')
zip_ref.close()

## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [6]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
train_horse_dir= os.path.join('/content/horse-or-human/horse-or-human/train/horses')
train_human_dir= os.path.join('/content/horse-or-human/horse-or-human/train/humans')
test_horse_dir= os.path.join('/content/horse-or-human/horse-or-human/validation/horses')
test_human_dir= os.path.join('/content/horse-or-human/horse-or-human/validation/humans')

In [14]:
print('Total train horse images: ',len(os.listdir(train_horse_dir)))
print('Total train human images: ',len(os.listdir(train_human_dir)))
print('Total test horse images: ',len(os.listdir(test_horse_dir)))
print('Total test human images: ',len(os.listdir(test_human_dir)))

Total train horse images:  500
Total train human images:  527
Total test horse images:  128
Total test human images:  128


**Model Building**

In [9]:
model= keras.Sequential([
    keras.layers.Conv2D(16,(3,3), activation='relu', input_shape=(300,300,3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(32,(3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64,(3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 78400)             0

In [11]:
from tensorflow.keras.optimizers import RMSprop

#compiling the model
model.compile(loss='binary_crossentropy',
             optimizer=RMSprop(lr=0.001),
             metrics=['acc'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen= ImageDataGenerator(rescale= 1./255)
train_gen= train_datagen.flow_from_directory( '/content/horse-or-human/horse-or-human/train',
                                            target_size=(300,300),
                                            batch_size=128,
                                            class_mode='binary')

test_datagen= ImageDataGenerator(rescale= 1./255)
test_gen= test_datagen.flow_from_directory( '/content/horse-or-human/horse-or-human/validation',
                                            target_size=(300,300),
                                            batch_size=32,
                                            class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [18]:
model.fit_generator( train_gen,
                   steps_per_epoch=8,
                   epochs=15,
                   validation_data= test_gen,
                   validation_steps= 8,
                   verbose=2 )

Epoch 1/15
8/8 - 11s - loss: 5.6544 - acc: 0.5506 - val_loss: 1.0487 - val_acc: 0.5000
Epoch 2/15
8/8 - 6s - loss: 0.8124 - acc: 0.6418 - val_loss: 0.6454 - val_acc: 0.6055
Epoch 3/15
8/8 - 6s - loss: 0.6741 - acc: 0.7564 - val_loss: 1.1715 - val_acc: 0.7031
Epoch 4/15
8/8 - 6s - loss: 0.2167 - acc: 0.9221 - val_loss: 1.2909 - val_acc: 0.7852
Epoch 5/15
8/8 - 6s - loss: 0.1140 - acc: 0.9600 - val_loss: 1.2694 - val_acc: 0.8164
Epoch 6/15
8/8 - 6s - loss: 0.0446 - acc: 0.9844 - val_loss: 0.7857 - val_acc: 0.8906
Epoch 7/15
8/8 - 7s - loss: 0.5487 - acc: 0.8633 - val_loss: 0.6247 - val_acc: 0.8516
Epoch 8/15
8/8 - 6s - loss: 0.2763 - acc: 0.8854 - val_loss: 1.3920 - val_acc: 0.8516
Epoch 9/15
8/8 - 6s - loss: 0.0404 - acc: 0.9878 - val_loss: 1.8434 - val_acc: 0.8320
Epoch 10/15
8/8 - 6s - loss: 0.0199 - acc: 0.9900 - val_loss: 1.8299 - val_acc: 0.8398
Epoch 11/15
8/8 - 7s - loss: 0.0070 - acc: 1.0000 - val_loss: 2.3084 - val_acc: 0.8359
Epoch 12/15
8/8 - 7s - loss: 0.0037 - acc: 1.0000 -

In [0]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image

**Prediction on unknown images**

In [21]:
uploaded= files.upload()

for fn in uploaded.keys():
  # predict image
  path='/content/' + fn
  img= image.load_img(path, target_size=(300,300))
  x= image.img_to_array(img)
  x= np.expand_dims(x, axis=0)
  
  images= np.vstack([x])
  classes= model.predict(images, batch_size=10)
  print(classes[0])
  if(classes[0]>0.5):
    print(fn + ' is a human')
    
  else:
    print(fn + ' is a horse')
  

Saving ae8a05_88e7d73b8d7442b6afb3886681b36fda_mv2.jpg to ae8a05_88e7d73b8d7442b6afb3886681b36fda_mv2.jpg
Saving download (1).jpg to download (1).jpg
Saving download (2).jpg to download (2).jpg
Saving download (3).jpg to download (3).jpg
Saving download (4).jpg to download (4).jpg
Saving download.jpg to download.jpg
Saving hairremoval-960x580.jpg to hairremoval-960x580.jpg
Saving photo-1450052590821-8bf91254a353.jpg to photo-1450052590821-8bf91254a353.jpg
[1.]
ae8a05_88e7d73b8d7442b6afb3886681b36fda_mv2.jpg is a human
[1.]
download (1).jpg is a human
[1.]
download (2).jpg is a human
[1.]
download (3).jpg is a human
[1.]
download (4).jpg is a human
[1.]
download.jpg is a human
[1.]
hairremoval-960x580.jpg is a human
[1.]
photo-1450052590821-8bf91254a353.jpg is a human
